In [19]:
from cogdl.datasets.grb_data import Cora_GRBDataset
from cogdl.utils import set_random_seed
from cogdl.utils.grb_utils import evaluate, GCNAdjNorm
import copy
import torch
dataset = Cora_GRBDataset()
graph = copy.deepcopy(dataset.get(0))
# device = "cpu"
device = "cuda:0"
device_ids = [0]
graph.to(device)
test_mask = graph.test_mask
set_random_seed(40)

In [20]:
# train surrogate model
from cogdl.models.nn import GCN
from cogdl.trainer import Trainer
from cogdl.wrappers import fetch_model_wrapper, fetch_data_wrapper
model_sur = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=2,
    dropout=0.5,
    activation="relu"
)
print(model_sur)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_sur, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_sur.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_sur.to(device)
test_score = evaluate(model_sur,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for surrogate model: {:.4f}.".format(test_score))

GCN(
  (layers): ModuleList(
    (0): GCNLayer(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (act): ReLU(inplace=True)
    )
    (1): GCNLayer(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)
Model Parameters: 19847


Epoch: 200, train_loss:  0.5650, val_acc:  0.8060: 100%|██████████| 200/200 [00:03<00:00, 64.09it/s]


Saving 174-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.8171641791044776, 'val_acc': 0.8246268656716418}
Test score before attack for surrogate model: 0.8172.


In [21]:
# train target model

model_target = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=3,
    dropout=0.5,
    activation=None,
    norm="layernorm"
)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_target, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_target.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_target.to(device)
test_score = evaluate(model_target,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for target model: {:.4f}.".format(test_score))

Model Parameters: 24263


Epoch: 116, train_loss:  0.4251, val_acc:  0.7948:  58%|█████▊    | 116/200 [00:02<00:01, 42.19it/s]


Saving 67-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.8196517412935324, 'val_acc': 0.8097014925373134}
Test score before attack for target model: 0.8197.


In [5]:
# train target model

from cogdl.models.nn import SAGE

model_target = SAGE(
    graph.num_features,
    graph.num_classes,
    hidden_size=128,
    num_layers=2,
    dropout=0.5,
    norm=None
)
print(model_target)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_target, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_target.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_target.to(device)
test_score_target_raw = evaluate(model_target,
                                 graph,
                                 mask=test_mask,
                                 device=device)
print("Test score before attack for target model: {:.4f}.".format(test_score_target_raw))

Model Parameters: 79239


Epoch: 681, train_loss:  1.4261, val_acc:  0.7575:  34%|███▍      | 681/2000 [00:24<00:46, 28.12it/s]


Saving 182-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.7985074626865671, 'val_acc': 0.832089552238806}
Test score before attack for target model: 0.6604.


In [22]:
# FGSM attack
from cogdl.attack.injection import FGSM
attack = FGSM(epsilon=0.01,
              n_epoch=1000,
              n_inject_max=100,
              n_edge_max=200,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

: 

In [10]:
from cogdl.attack.injection import PGD
attack = PGD(epsilon=0.01,
              n_epoch=1000,
              n_inject_max=100,
              n_edge_max=200,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Epoch 999, Loss: 2.2342, Surrogate test score: 0.0771: 100%|██████████| 1000/1000 [00:35<00:00, 28.44it/s]


Surrogate test score: 0.0771
Attack runtime: 35.4433.
Graph(x=[2680, 302], y=[2680], train_mask=[2680], val_mask=[2680], test_mask=[2680], edge_index=[2, 10296], edge_weight=[10296], edge_attr=[10296])
Graph(x=[2780, 302], y=[2680], edge_index=[2, 50296], edge_attr=[50296])
Graph(x=[2780, 302], y=[2680], edge_index=[2, 50296], edge_weight=[50296], edge_attr=[50296])
Test score after attack for surrogate model: 0.2786.
Test score after attack for target model: 0.1816.


In [11]:
from cogdl.attack.injection import RAND
attack = RAND(n_inject_max=100,
              n_edge_max=200,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Loss: -1.8378, Surrogate test acc: 0.6965
Attack runtime: 0.1471.
Test score after attack for surrogate model: 0.3818.
Test score after attack for target model: 0.3122.


In [12]:
from cogdl.attack.injection import SPEIT
attack = SPEIT(lr=0.02,
               n_epoch=1000,
               n_inject_max=100,
               n_edge_max=200,
               feat_lim_min=-1,
               feat_lim_max=1,
               device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Epoch 999, Loss: -2.2334, Surrogate test score: 0.0771: 100%|██████████| 1000/1000 [00:33<00:00, 29.81it/s]


Surrogate test score: 0.0771
Attack runtime: 33.6856.
Test score after attack for surrogate model: 0.2711.
Test score after attack for target model: 0.2413.


In [8]:
from cogdl.attack.injection import TDGIA
attack = TDGIA(lr=0.01,
               n_epoch=1000,
               n_inject_max=100,
               n_edge_max=200,
               feat_lim_min=-1,
               feat_lim_max=1,
               device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print(graph_attack)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Attacking: Sequential inject 20/100 nodes


Epoch 999, Loss: 6.8068, Surrogate test score: 0.7363: 100%|██████████| 1000/1000 [00:17<00:00, 56.66it/s]


Attacking: Sequential inject 40/100 nodes


Epoch 999, Loss: 6.3547, Surrogate test score: 0.6978: 100%|██████████| 1000/1000 [00:18<00:00, 53.52it/s]


Attacking: Sequential inject 60/100 nodes


...

Attacking: Sequential inject 80/100 nodes


Epoch 999, Loss: 5.9993, Surrogate test score: 0.6816: 100%|██████████| 1000/1000 [00:22<00:00, 44.09it/s]


Attacking: Sequential inject 100/100 nodes


Epoch 999, Loss: 5.8305, Surrogate test score: 0.6729: 100%|██████████| 1000/1000 [00:23<00:00, 43.44it/s]


Attack runtime: 104.0285.
Graph(x=[2680, 302], y=[2680], train_mask=[2680], val_mask=[2680], test_mask=[2680], edge_index=[2, 10296], edge_weight=[10296], edge_attr=[10296])
Graph(x=[2780, 302], y=[2680], edge_index=[2, 21096], edge_attr=[21096])
Graph(x=[2780, 302], y=[2680], edge_index=[2, 21096], edge_weight=[21096], edge_attr=[21096])
Test score after attack for surrogate model: 0.5672.
Test score after attack for target model: 0.7065.
